In [ ]:
import datacube
from datacube.utils import cog
from datetime import datetime
from datetime import timedelta
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from datacube.utils.masking import make_mask
from datacube.utils.masking import mask_invalid_data
from odc.algo import mask_cleanup
import sys
sys.path.insert(1, '../Tools/')
from dea_tools.datahandling import load_ard
from dea_tools.plotting import rgb, display_map
from dea_tools.bandindices import calculate_indices
from dea_tools.dask import create_local_dask_cluster
from dea_tools.spatial import xr_vectorize
import scipy.ndimage
# Create local dask cluster to improve data load time
client = create_local_dask_cluster(return_client=True)
import scipy.ndimage
import xarray
import numpy
import datacube
from datacube.utils.masking import make_mask
from datacube.utils.masking import mask_invalid_data
from odc.algo import mask_cleanup

import sys
sys.path.insert(1, '../Tools/')
from dea_tools.plotting import rgb

In [ ]:
dc = datacube.Datacube(app="bais2_mapping")

In [ ]:
# Define the area of interest
latitude = (-28.1218076, -28.2168763)
longitude = (153.1025233, 153.2382923)

# Set the range of dates for the complete sample
time = ('1999', '2024')
# Compute the bounding box for the study area
study_area_lat = (-28.1218076, -28.2168763)
study_area_lon = (153.1025233, 153.2382923)

display_map(x=study_area_lon, y=study_area_lat, margin=-0.2)

In [ ]:
# Fire event date
fire_date = '2019-09-06'

# Length of baseline period
fire_length = '3 months'

In [ ]:
# Define dates for loading data
if fire_length == '12 months':
    time_step = timedelta(days=365)
if fire_length == '6 months':
    time_step = timedelta(days=182.5)
if fire_length == '3 months':
    time_step = timedelta(days=91)

# Calculate the start and end date for baseline data load
start_date_pre = datetime.strftime(
    ((datetime.strptime(fire_date, '%Y-%m-%d')) - time_step), '%Y-%m-%d')
end_date_pre = datetime.strftime(
    ((datetime.strptime(fire_date, '%Y-%m-%d')) - timedelta(days=1)),
    '%Y-%m-%d')

# Calculate end date for post fire data load
start_date_post = datetime.strftime(
    ((datetime.strptime(fire_date, '%Y-%m-%d')) + timedelta(days=15)),
    '%Y-%m-%d')
end_date_post = datetime.strftime(
    ((datetime.strptime(fire_date, '%Y-%m-%d')) + timedelta(days=90)),
    '%Y-%m-%d')


# Print dates
print(f'start_date_pre:  {start_date_pre}')
print(f'end_date_pre:    {end_date_pre}')
print(f'fire_date:       {fire_date}')
print(f'start_date_post: {start_date_post}')
print(f'end_date_post:   {end_date_post}')

In [ ]:
# Band List
#dc_measurements = dc.list_measurements()
#dc_measurements.loc[['ga_s2am_ard_3']]

In [ ]:
resolution = (-20, 20)
measurements = ['nbart_red_edge_2', 'nbart_red_edge_3', 'nbart_nir_2', 'nbart_red', 'nbart_swir_3', 'nbart_nir_1', 'nbart_green', 'nbart_blue','fmask' ]
min_gooddata = 0.9
output_crs = 'EPSG:9473'

In [ ]:
# Load all data in baseline period available from ARD data
dsbaseline = load_ard(dc=dc,
                        products=['ga_s2am_ard_3', 'ga_s2bm_ard_3'],
                        x=study_area_lon,
                        y=study_area_lat,
                        time=(start_date_pre, end_date_pre),
                        measurements=measurements,
                        min_gooddata=min_gooddata,
                        resampling={
                         "fmask": "nearest",
                          "*": "bilinear"},
                        output_crs=output_crs,
                        resolution=resolution,
                        group_by='solar_day')


In [ ]:
#selecting best timestep
rgb(dsbaseline,col="time")

In [ ]:
dsbaseline.fmask.attrs["flags_definition"]

In [ ]:
dsbaseline.fmask.plot(col="time", col_wrap=4)

In [ ]:
# Create the mask based on "valid" pixels
clear_mask = make_mask(dsbaseline.fmask, fmask="valid")
#clear_mask.plot(col="time", col_wrap=4)

In [ ]:
# Apply the mask
baseclear = dsbaseline.where(clear_mask)
#rgb(baseclear, col="time")

In [ ]:
%matplotlib inline

import datacube
import geopandas as gpd
import odc.geo.xr
from odc.geo.geom import Geometry

import sys

sys.path.insert(1, "../Tools/")
from dea_tools.plotting import rgb

In [ ]:
polygon_to_drill = (
    "lamington_boundary.geojson"
)
# Read vector file
polygon_to_drill = gpd.read_file(polygon_to_drill)

# Check that the polygon loaded as expected. We'll just print the first 3 rows to check
#polygon_to_drill.head(3)
# Select polygon
shapely_geometry = polygon_to_drill.iloc[0].geometry

# Convert to Geometry object with CRS information
geom = Geometry(geom=shapely_geometry, crs=polygon_to_drill.crs)
geom

In [ ]:
# Mask out all pixels outside of our polygon:
base_fullymasked = baseclear.odc.mask(poly=geom)
#rgb(base_fullymasked, col="time")

In [ ]:
# Set the timesteps to visualise
best_timestep = 19

# Generate RGB plot of best timestamp
rgb(base_fullymasked, index=[best_timestep])

In [ ]:
#defining bais2 calculation
B06=base_fullymasked.nbart_red_edge_2/10000
B07=base_fullymasked.nbart_red_edge_3/10000
B8A=base_fullymasked.nbart_nir_2/10000
B04=base_fullymasked.nbart_red/10000
B12=base_fullymasked.nbart_swir_3/10000

#applying the calculation to the bands in ds then adding it as a variable in that ds
base_fullymasked['BAIS2'] = (1-((B06*B07*B8A)/B04)**0.5)*((B12-B8A)/((B12+B8A)**0.5)+1)

In [ ]:
#selecting the baseline timestep
baseline_BAI = base_fullymasked.BAIS2.isel(time=19)
# Select RGB baseline
baseline_image = base_fullymasked.isel(time=19)

In [ ]:
#plotting histogram of baseline_BAI, used to set vmin and vmax
plt.hist(baseline_BAI.data.flatten(),bins=np.arange(-1,1,0.1))

In [ ]:
#Plotting the baseline comparison
# Set up subplots for baseline
f, axarr = plt.subplots(1, 2, figsize=(15, 7), squeeze=False)

# Visualise post-fire image as a true colour image
rgb(baseline_image, 
    bands=['nbart_red', 'nbart_green', 'nbart_blue'], 
    ax=axarr[0, 0])
axarr[0, 0].set_title('Pre-fire RGB')
axarr[0, 0].set_xlabel('X coordinate')
axarr[0, 0].set_ylabel('Y coordinate')

# Visualise post-fire image as BAI image
baseline_BAI.plot(cmap='RdBu', vmin=-1, vmax=1, ax=axarr[0, 1])
axarr[0, 1].set_title('Pre-fire BAIS2')
axarr[0, 1].yaxis.set_visible(True)
axarr[0, 1].set_xlabel('X coordinate');



In [ ]:
#Preparing the post-fire images
#Load post-fire data from Sentinel-2A and 2B
# Load all data in baseline period available from ARD data
dspost = load_ard(dc=dc,
                        products=['ga_s2am_ard_3', 'ga_s2bm_ard_3'],
                        x=study_area_lon,
                        y=study_area_lat,
                        time=('2019-12-5', '2020-01-30'),
                        measurements=measurements,
                        min_gooddata=min_gooddata,
                        resampling={
                         "fmask": "nearest",
                          "*": "bilinear"},
                        output_crs=output_crs,
                        resolution=resolution,
                        group_by='solar_day')
# Load all data in baseline period available from ARD data


In [ ]:
#selecting post fire image
rgb(dspost,col="time")

In [ ]:
dspost.fmask.attrs["flags_definition"]

In [ ]:
clear_mask_post = make_mask(dspost.fmask, fmask="valid")
#clear_mask.plot(col="time", col_wrap=4)

In [ ]:
postclear = dspost.where(clear_mask_post)

In [ ]:
# Mask out all pixels outside of our polygon:
post_fullymasked = postclear.odc.mask(poly=geom)
#rgb(post_fullymasked, col="time")

In [ ]:
# Set the timesteps to visualise
best_timestep1 = 6

# Generate RGB plot of best timestamp
rgb(post_fullymasked, index=[best_timestep1])

In [ ]:
#defining bais2 calculation
B06=post_fullymasked.nbart_red_edge_2/10000
B07=post_fullymasked.nbart_red_edge_3/10000
B8A=post_fullymasked.nbart_nir_2/10000
B04=post_fullymasked.nbart_red/10000
B12=post_fullymasked.nbart_swir_3/10000

#applying the calculation to the bands in ds then adding it as a variable in that ds
post_fullymasked['BAIS2'] = (1-((B06*B07*B8A)/B04)**0.5)*((B12-B8A)/((B12+B8A)**0.5)+1)

In [ ]:
post_fullymasked

In [ ]:
#selecting the baseline timestep
post_BAI = post_fullymasked.BAIS2.isel(time=best_timestep1)
# Select RGB baseline
post_image = post_fullymasked.isel(time=best_timestep1)

In [ ]:
#post_BAI.data.flatten()
#plotting histogram of baseline_BAI, used to set vmin and vmax
plt.hist(post_BAI.data.flatten(),bins=np.arange(-1,1.4,0.1))

In [ ]:
# Set up subplots
f, axarr = plt.subplots(1, 2, figsize=(15, 7), squeeze=False)

# Visualise post-fire image as a true colour image
rgb(post_image, 
    bands=['nbart_red', 'nbart_green', 'nbart_blue'], 
    ax=axarr[0, 0])
axarr[0, 0].set_title('Post-fire RGB')
axarr[0, 0].set_xlabel('X coordinate')
axarr[0, 0].set_ylabel('Y coordinate')

# Visualise post-fire image as BAI image
post_BAI.plot(cmap='RdBu', vmin=0.5, vmax=0.8, ax=axarr[0, 1])
axarr[0, 1].set_title('Post-fire BAIS2')
axarr[0, 1].yaxis.set_visible(True)
axarr[0, 1].set_xlabel('X coordinate');


In [ ]:
#creating delta of baseline and post images
delta_BAI = post_BAI - baseline_BAI

# Visualise dBAIS2 image
delta_BAI.plot(cmap='RdBu_r', vmin=-0.3, vmax=0.6, figsize=(11, 9))
plt.xlabel('X coordinate')
plt.ylabel('Y coordinate');

In [ ]:
#plot histogram
plt.hist(delta_BAI.data.flatten(),bins=np.arange(-0.75,0.6,0.1))


In [ ]:
bam = delta_BAI
# Create a figure to plot the chosen fire severity index
f, axarr = plt.subplots(
    1, 2, figsize=(15, 10), squeeze=False, gridspec_kw={"width_ratios": [1, 5]}
)

# Calculate and round the dBAI dataarray value range to set determine the plots colourmap range
bam_BAI_min = round(float(bam.quantile(0.005)), 1)
bam_BAI_max = round(float(bam.quantile(0.995)), 1)

# PLot the dBAI dataarray on the second subplot of the above figure
bam.plot(cmap="RdBu_r", vmin=bam_BAI_min, vmax=bam_BAI_max, ax=axarr[(0, 1)])

# Plot a histogram of dBAI values in the first figure subplot.
# Calculate a colourmap from the dataarray plot by iterating through individual histogram patches
cm = plt.colormaps.get_cmap("RdBu_r")

n, bins, patches = xr.plot.hist(
    darray=bam,
    bins=np.arange(bam_BAI_min, bam_BAI_max + 0.05, 0.05),
    align="mid",
    orientation="horizontal",
    ec="black",
    yticks=(np.arange(bam_BAI_min - 0.05, bam_BAI_max + 0.05, step=0.05)),
    ax=axarr[(0, 0)],
)

# Match the colour scale of the histogram to that used in the map plot.
bin_centers = 0.5 * (bins[:-1] + bins[1:])
col = bin_centers - min(bin_centers)
col /= max(col)
for c, p in zip(col, patches):
    plt.setp(p, "facecolor", cm(c))

# Set titles for each subplot
axarr[0, 0].set_title('dBAIS2' + " Histogram")
axarr[0, 1].set_xlabel('X Coordinate')
axarr[0, 1].set_ylabel('Y Coordinate')
axarr[0, 1].set_title(
    'dBAIS2'
    + " measured between "
    + str(baseline_BAI.time.values)[:10]
    + " - "
    + str(post_BAI.time.values)[:10]
)

# Set the x-axis label and number of x-axis ticks for the histogram plot
axarr[0, 0].set_xlabel('dBAIS2' + " count")
axarr[0, 0].xaxis.set_major_locator(plt.MaxNLocator(3))

In [ ]:
#create pixel classes
unburnt = delta_BAI < 0.2
low = (delta_BAI > 0.2) & (delta_BAI < 0.3)
moderate = (delta_BAI > 0.3) & (delta_BAI < 0.4)
high = (delta_BAI > 0.4) & (delta_BAI < 0.45)
extreme = delta_BAI > 0.45

In [ ]:
import xarray as xr

# Assuming 'delta_BAI' is a NumPy array
#delta_BAI = xr.DataArray(delta_BAI, dims=['636', '739'])  # Replace 'dim1', 'dim2' with actual dimension names

In [ ]:
# Multiply by 1000 to match the classification ranges
BAI_scaled = delta_BAI * 1000

# Define the conditions and corresponding values using EU classes (https://forest-fire.emergency.copernicus.eu/about-effis/technical-background/fire-severity)
conditions = [
    (BAI_scaled < 200),
    (BAI_scaled >= 200) & (BAI_scaled < 300),
    (BAI_scaled >= 300) & (BAI_scaled < 400),
    (BAI_scaled >= 400) & (BAI_scaled < 450),
    (BAI_scaled >= 450)
]

values = [1, 2, 3, 4, 5]

# Create a new DataArray with the classified values
BAI_classified = xr.DataArray(
    np.select(conditions, values),
    coords=delta_BAI.coords,
    dims=delta_BAI.dims,
    name='BAI_Class'
)

# Add attributes for class descriptions
BAI_classified.attrs['classes'] = {
    1: 'Unburned or Regrowth',
    2: 'Low severity',
    3: 'Moderate severity',
    4: 'High severity',
    5: 'Extreme severity'
}

In [ ]:
pip install matplotlib-scalebar

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
from matplotlib_scalebar.scalebar import ScaleBar

# Assuming you have already created BAI_classified as in the previous example

# Define colors for each class
colors = ['olivedrab', 'yellow', 'orange', 'red', 'darkred']
cmap = ListedColormap(colors)

# Set up the colorbar ticks and boundaries
bounds = np.arange(0.5, 6.5, 1)  # 0.5, 1.5, 2.5, 3.5, 4.5, 5.5
norm = BoundaryNorm(bounds, cmap.N)

# Create the plot
fig, ax = plt.subplots(figsize=(12, 10))

# Plot the data
im = ax.imshow(BAI_classified, cmap=cmap, norm=norm)

# Add colorbar
cbar = fig.colorbar(im, ax=ax, orientation='vertical', pad=0.02)

# Set colorbar ticks and labels
cbar.set_ticks(np.arange(1, 6))
cbar.set_ticklabels(['Unburned or Regrowth', 'Low severity', 'Moderate severity', 
                     'High severity', 'Extreme severity'])

# Set title and labels
ax.set_title('Burn Severity Map (dBAIS2)', fontsize=16)

# Remove x and y ticks
ax.set_xticks([])
ax.set_yticks([])

# Add a scale bar
scalebar = ScaleBar(1, location='lower right')  # You can adjust the value to represent the map scale
ax.add_artist(scalebar)

# Add a north arrow
# Define the position and size of the north arrow
arrow_x = 0.9  # x position (in axis coordinates, from 0 to 1)
arrow_y = 0.9  # y position (in axis coordinates, from 0 to 1)
arrow_length = 0.1  # Length of the arrow in axis coordinates

x, y, arrow_length = 0.05, 0.95, 0.07
ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
            arrowprops=dict(facecolor='black', width=5, headwidth=15),
            ha='center', va='center', fontsize=20,
            xycoords=ax.transAxes)

plt.tight_layout()
plt.show()

In [ ]:
# Constants for calculating burnt area (using your existing constants)
pixel_length = resolution[1]  # in metres
m_per_km = 1000  # conversion from metres to kilometres
area_per_pixel = pixel_length ** 2 / m_per_km ** 2

# Initialize dictionary to store results
severity_stats = {}

# Calculate area for each class
for class_value, class_name in BAI_classified.attrs['classes'].items():
    # Calculate area for this class
    class_pixels = (BAI_classified == class_value).sum()
    class_area = class_pixels * area_per_pixel
    
    # Store in dictionary
    severity_stats[class_name] = class_area.item()

# Calculate total area (excluding NaN)
total_valid_area = sum(severity_stats.values())

# Create and print formatted results
print("\nBurn Severity Statistics:")
print("-" * 60)
print(f"{'Severity Class':<25} {'Area (km²)':<15} {'Percentage':>10}")
print("-" * 60)

for class_name, area in severity_stats.items():
    percentage = (area / total_valid_area) * 100
    print(f"{class_name:<25} {area:>8.2f} km²    {percentage:>8.1f}%")

print("-" * 60)
print(f"{'Total area':<25} {total_valid_area:>8.2f} km²    {100:>8.1f}%")

# Optional: Calculate NaN area
nan_area = BAI_classified.isnull().sum() * area_per_pixel
print(f"{'No data (NaN) area':<25} {nan_area.item():>8.2f} km²")

# Previous code remains the same until the results section

# Calculate total burnt area (excluding unburned class)
total_burnt = sum(area for class_name, area in severity_stats.items() 
                 if class_name != 'Unburned or Regrowth')

print("-" * 60)
print(f"{'Total area':<25} {total_valid_area:>8.2f} km²    {100:>8.1f}%")
print(f"{'No data (NaN) area':<25} {nan_area.item():>8.2f} km²")
print("-" * 60)
print(f"{'Total burnt area':<25} {total_burnt:>8.2f} km²    {(total_burnt/total_valid_area)*100:>8.1f}%")

In [ ]:
# First, let's create a comparison
comparison = xr.Dataset({
    'simple_burnt': burnt,
    'severity_burnt': BAI_classified > 1  # All classes except unburned
})

# Calculate areas for comparison
simple_burnt_area = burnt.sum() * area_per_pixel
severity_burnt_area = (BAI_classified > 1).sum() * area_per_pixel

# Calculate the difference map
difference = comparison.simple_burnt != (BAI_classified > 1)
difference_area = difference.sum() * area_per_pixel

print("Comparison of burnt area calculations:")
print(f"Simple threshold method: {simple_burnt_area.item():.2f} km²")
print(f"Severity classes method: {severity_burnt_area.item():.2f} km²")
print(f"Area of disagreement:    {difference_area.item():.2f} km²")

# Let's see where these values differ
threshold_scaled = threshold * 1000  # Your original threshold scaled
print(f"\nThreshold analysis:")
print(f"Original threshold: {threshold}")
print(f"Scaled threshold:  {threshold_scaled}")

In [ ]:
# Align the thresholds
aligned_threshold = 0.2  # Your original threshold

# For burn area calculation
burnt = delta_BAI > aligned_threshold

# For severity calculation (using aligned thresholds)
BAI_scaled = delta_BAI * 1000

# Define the conditions ensuring alignment with burn area
conditions = [
    (BAI_scaled <= aligned_threshold * 1000),  # Unburned
    (BAI_scaled > aligned_threshold * 1000) & (BAI_scaled < 300),
    (BAI_scaled >= 300) & (BAI_scaled < 400),
    (BAI_scaled >= 400) & (BAI_scaled < 450),
    (BAI_scaled >= 450)
]

values = [1, 2, 3, 4, 5]

# Create aligned severity classification
BAI_classified_aligned = xr.DataArray(
    np.select(conditions, values),
    coords=delta_BAI.coords,
    dims=delta_BAI.dims,
    name='BAI_Class'
)

In [ ]:
#masking the final burn scar

# Set threshold
threshold = 0.2


# Apply threshold
burnt = delta_BAI > threshold

delta_BAI['burnt'] = delta_BAI < threshold
#delta_BAIt = delta_BAI.burnt


total_burn = delta_BAI.where(burnt==1)
# Visualise dBAI image

total_burn.plot(cmap='inferno', figsize=(11, 9), vmax=0.6, vmin=-0.2)
plt.title('dBAIS2 Burn Area')
plt.xlabel('X coordinate')
plt.ylabel('Y coordinate');

In [ ]:
from scipy import ndimage
from skimage import morphology

sys.path.insert(1, "../Tools/")
from dea_tools.bandindices import calculate_indices
from dea_tools.datahandling import load_ard
from dea_tools.plotting import display_map, rgb, plot_variable_images
from dea_tools.spatial import xr_rasterize, xr_vectorize

In [ ]:
#salt/pepper clean?


In [ ]:
#Calculating Burnt Area

In [ ]:
# Constants for calculating burnt area
pixel_length = resolution[1]  # in metres
m_per_km = 1000  # conversion from metres to kilometres

# Area per pixel
area_per_pixel = pixel_length ** 2 / m_per_km ** 2

# Calculate areas
unburnt_area = (delta_BAI <= threshold).sum() * area_per_pixel
burnt_area = burnt.sum() * area_per_pixel
not_nan_area = delta_BAI.notnull().sum() * area_per_pixel
nan_area = delta_BAI.isnull().sum() * area_per_pixel
total_area = unburnt_area + burnt_area + nan_area

print(f'Unburnt area:            {unburnt_area.item():.2f} km^2')
print(f'Burnt area:              {burnt_area.item():.2f} km^2')
print(f'Nan area:                {nan_area.item():.2f} km^2')
print(f'Total area (no nans):    {not_nan_area.item():.2f} km^2')
print(f'Total area (with nans):  {total_area.item():.2f} km^2')

In [ ]:
# Constants for calculating burnt area
pixel_length = resolution[1]  # in metres
m_per_km = 1000  # conversion from metres to kilometres

# Area per pixel
area_per_pixel = pixel_length ** 2 / m_per_km ** 2

# Calculate burnt area (only where burnt == 1)
burnt_pixels = burnt.sum()
burnt_area = burnt_pixels * area_per_pixel

# Calculate total area and unburnt area for context
total_pixels = burnt.notnull().sum()
total_area = total_pixels * area_per_pixel
unburnt_area = (total_area - burnt_area)

print("\nBurn Area Statistics:")
print("-" * 40)
print(f"Total burnt area:        {burnt_area.item():.2f} km²")
print(f"Total unburnt area:      {unburnt_area.item():.2f} km²")
print(f"Total area:              {total_area.item():.2f} km²")
print(f"Percentage burnt:        {(burnt_area.item()/total_area.item()*100):.1f}%")

# Optional: Add number of pixels for verification
print("\nPixel Counts:")
print(f"Burnt pixels:            {burnt_pixels.item():,}")
print(f"Total valid pixels:      {total_pixels.item():,}")

In [ ]:
#download as shapefile
from dea_tools.spatial import xr_rasterize, xr_vectorize
# Convert the burnt area from raster to vector format
gdf = xr_vectorize(
    da=total_burn, output_path= "total_me.shp"
)

In [ ]:
#download as shapefile
from dea_tools.spatial import xr_rasterize, xr_vectorize
# Convert the burnt area from raster to vector format
gdf = xr_vectorize(
    da=BAI_classified, output_path= "sevvv.shp"
)

In [ ]:
#vectorise
#polygons = xr_vectorize(total_burn)
#polygons["area"] = polygons.area
#polygons["size"] = np.where(polygons["area"] < 50000, 1, 2)  # se

In [ ]:
# Set specific EPSG code (replace 32754 with your correct EPSG code)
total_burn.rio.write_crs('EPSG:9473', inplace=True)

# Then export
write_cog(geo_im=total_burn,
          fname='tburnnnn.tif',
          overwrite=True,
          nodata=0.0)


In [ ]:
#export burnt area map to test
from datacube.utils.cog import write_cog
# Write GeoTIFF
write_cog(geo_im=total_burn,
          fname='tburn.tif',
          overwrite=True,
          nodata=0.0)

In [ ]:
# Import required library
import geopandas as gpd

# Read the shapefile
qpws = gpd.read_file('qpws_layer.shp')

# Print basic information about the layer
print("QPWS Layer Information:")
print("-" * 40)
print(f"CRS: {qpws.crs}")
print(f"Number of features: {len(qpws)}")
print(f"Columns available: {qpws.columns.tolist()}")

# Calculate areas in square kilometers
qpws['area_km2'] = qpws.geometry.area / 1_000_000  # converts m² to km²

# Calculate total area
total_area = qpws['area_km2'].sum()

print("\nArea Statistics:")
print("-" * 40)
print(f"Total area: {total_area:.2f} km²")

# If you have multiple features and want to see individual areas:
if len(qpws) > 1:
    print("\nIndividual Feature Areas:")
    for idx, area in enumerate(qpws['area_km2']):
        print(f"Feature {idx + 1}: {area:.2f} km²")


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd

# Read the shapefile
qpws = gpd.read_file('qpws_layer.shp')

# Project to EPSG:9473
qpws_projected = qpws.to_crs(epsg=9473)

# Calculate areas in square kilometers
qpws_projected['area_km2'] = qpws_projected.geometry.area / 1_000_000

# Print basic information
print("QPWS Layer Information:")
print("-" * 40)
print(f"CRS: {qpws_projected.crs}")
print(f"Number of features: {len(qpws_projected)}")
print(f"Columns available: {qpws_projected.columns.tolist()}")

# Area Statistics
print("\nArea Statistics:")
print("-" * 40)
print(f"Total area: {qpws_projected['area_km2'].sum():.2f} km²")

# Severity Statistics
print("\nSeverity Statistics:")
print("-" * 40)
severity_stats = qpws_projected.groupby('Severity').agg({
    'area_km2': ['sum', 'count']
}).round(2)
severity_stats.columns = ['Total Area (km²)', 'Number of Features']
print(severity_stats)

# Individual feature details
print("\nDetailed Feature Information:")
print("-" * 40)
for idx, row in qpws_projected.iterrows():
    print(f"Feature {idx + 1}:")
    print(f"Estate Name: {row['ESTATENAME']}")
    print(f"Severity: {row['Severity']}")
    print(f"Area: {row['area_km2']:.2f} km²")
    print("-" * 20)

# Visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7))

# Map of areas colored by severity
qpws_projected.plot(column='Severity', 
                   legend=True,
                   legend_kwds={'title': 'Severity'},
                   cmap='YlOrRd',
                   ax=ax1)
ax1.set_title('QPWS Areas by Severity')
ax1.axis('off')

# Bar plot of areas by severity
severity_areas = qpws_projected.groupby('Severity')['area_km2'].sum()
severity_areas.plot(kind='bar', 
                   ax=ax2,
                   color='orangered')
ax2.set_title('Total Area by Severity')
ax2.set_xlabel('Severity')
ax2.set_ylabel('Area (km²)')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

# Optional: Save statistics to a CSV file
severity_stats.to_csv('severity_statistics.csv')

In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Project QPWS layer
qpws = gpd.read_file('qpws_layer.shp')
qpws_projected = qpws.to_crs(epsg=9473)

# Calculate QPWS (dNBR) area
dnbr_area_m2 = qpws_projected.geometry.area.sum()
dnbr_ha = dnbr_area_m2 / 10000  # m² to hectares
dnbr_km2 = dnbr_area_m2 / 1000000  # m² to km²

# Convert BAIS2 burnt area from km² to hectares
bais2_km2 = burnt_area.item()  # from your existing calculation
bais2_ha = bais2_km2 * 100  # convert km² to hectares

# Calculate statistics
abs_diff_ha = abs(bais2_ha - dnbr_ha)
rel_diff_percent = (abs_diff_ha / bais2_ha) * 100
agreement_ratio = min(bais2_ha, dnbr_ha) / max(bais2_ha, dnbr_ha)
mean_area_ha = np.mean([bais2_ha, dnbr_ha])
std_dev_ha = np.std([bais2_ha, dnbr_ha])
cv_percent = (std_dev_ha / mean_area_ha) * 100

# Create statistics DataFrame
stats = pd.DataFrame({
    'Statistic': [
        'BAIS2 burnt area',
        'QPWS (dNBR) burnt area',
        'Absolute Difference',
        'Relative Difference',
        'Agreement Ratio',
        'Mean Burnt Area',
        'Standard Deviation',
        'Coefficient of Variation (CV)'
    ],
    'Value': [
        f'{bais2_ha:.2f} hectares ({bais2_km2:.2f} km²)',
        f'{dnbr_ha:.2f} hectares ({dnbr_km2:.2f} km²)',
        f'{abs_diff_ha:.2f} hectares',
        f'{rel_diff_percent:.2f}%',
        f'{agreement_ratio:.4f}',
        f'{mean_area_ha:.2f} hectares',
        f'{std_dev_ha:.2f} hectares',
        f'{cv_percent:.2f}%'
    ]
})

# Display the statistics table
print("\nBurnt Area Statistics Comparison:")
print("-" * 80)
print(stats.to_string(index=False))

# Create a visual comparison
fig, ax1 = plt.subplots(figsize=(10, 6))

# Bar plot comparison
areas = [bais2_ha, dnbr_ha]
labels = ['BAIS2', 'QPWS (dNBR)']
bars = ax1.bar(labels, areas, color=['skyblue', 'orange'])
ax1.set_ylabel('Area (hectares)')
ax1.set_title('Burnt Area Comparison')

# Add value labels on top of bars
for bar in bars:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.2f} ha',
             ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Optional: Save to CSV
stats.to_csv('burnt_area_comparison_stats.csv', index=False)

In [ ]:
# Convert the QPWS data first
qpws_severity = qpws_projected.groupby('Severity').agg({
    'geometry': lambda x: sum(item.area for item in x)
}).reset_index()

qpws_severity['Area_km2'] = qpws_severity['geometry'] / 1000000  # convert m² to km²
qpws_total_area = qpws_severity['Area_km2'].sum()
qpws_severity['Percentage'] = (qpws_severity['Area_km2'] / qpws_total_area) * 100

# Get the area values from severity_stats
# From your output, we know this is the 'Total Area (km²)' series
area_series = severity_stats['Total Area (km²)']
my_severity_stats = pd.DataFrame({
    'Class': area_series.index,
    'Area_km2': area_series.values
})
my_severity_stats['Percentage'] = (my_severity_stats['Area_km2'] / my_severity_stats['Area_km2'].sum()) * 100

# Create comparison table
print("\nSeverity Comparison:")
print("-" * 80)
print("| Classification | BAIS2 |  | QPWS |  |")
print("|---------------|--------|------------|--------|------------|")
print("| Severity Class | Area (km²) | Percentage | Area (km²) | Percentage |")
print("|---------------|------------|------------|------------|------------|")

# Combine and align the classes
class_mapping = {
    'Extreme': 'Extreme',
    'High': 'High',
    'Moderate': 'Moderate',
    'Low': 'Low'
}

for _, row in my_severity_stats.iterrows():
    bais_class = row['Class']
    qpws_class = class_mapping.get(bais_class, bais_class)
    
    bais_area = row['Area_km2']
    bais_pct = row['Percentage']
    
    qpws_row = qpws_severity[qpws_severity['Severity'] == qpws_class]
    qpws_area = qpws_row['Area_km2'].iloc[0] if len(qpws_row) > 0 else 0
    qpws_pct = qpws_row['Percentage'].iloc[0] if len(qpws_row) > 0 else 0
    
    print(f"| {bais_class:<13} | {bais_area:>8.2f} | {bais_pct:>8.1f}% | {qpws_area:>8.2f} | {qpws_pct:>8.1f}% |")

total_bais = my_severity_stats['Area_km2'].sum()
print("|---------------|------------|------------|------------|------------|")
print(f"| Total | {total_bais:>8.2f} | 100.0% | {qpws_total_area:>8.2f} | 100.0% |")

# Visualization
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# BAIS2 Severity Distribution
my_severity_stats.plot(kind='bar', x='Class', y='Area_km2', ax=ax1)
ax1.set_title('BAIS2 Severity Distribution')
ax1.set_xlabel('Severity Class')
ax1.set_ylabel('Area (km²)')
ax1.tick_params(axis='x', rotation=45)

# QPWS Severity Distribution
qpws_severity.plot(kind='bar', x='Severity', y='Area_km2', ax=ax2)
ax2.set_title('QPWS Severity Distribution')
ax2.set_xlabel('Severity Class')
ax2.set_ylabel('Area (km²)')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Create the BAIS2 statistics DataFrame manually with standardized labels
bais2_data = {
    'Class': [
        'Low',           # Was 'Low severity'
        'Moderate',      # Was 'Moderate low severity'
        'High',          # Was 'Moderate high severity'
        'Extreme'        # Was 'High severity'
    ],
    'Area_km2': [7.28, 5.03, 1.31, 1.37],
    'Percentage': [48.6, 33.6, 8.7, 9.1]
}
my_severity_stats = pd.DataFrame(bais2_data)

# Calculate QPWS severity statistics
qpws_severity = qpws_projected.groupby('Severity').agg({
    'geometry': lambda x: sum(item.area for item in x)
}).reset_index()

qpws_severity['Area_km2'] = qpws_severity['geometry'] / 1000000  # convert m² to km²
qpws_total_area = qpws_severity['Area_km2'].sum()
qpws_severity['Percentage'] = (qpws_severity['Area_km2'] / qpws_total_area) * 100

# Ensure consistent ordering
severity_order = ['Low', 'Moderate', 'High', 'Extreme']
my_severity_stats = my_severity_stats.set_index('Class').reindex(severity_order).reset_index()
qpws_severity = qpws_severity.set_index('Severity').reindex(severity_order).reset_index()

# Create comparison table
print("\nSeverity Comparison:")
print("-" * 80)
print("| Classification | BAIS2 |  | QPWS |  |")
print("|---------------|--------|------------|--------|------------|")
print("| Severity Class | Area (km²) | Percentage | Area (km²) | Percentage |")
print("|---------------|------------|------------|------------|------------|")

for severity in severity_order:
    bais_row = my_severity_stats[my_severity_stats['Class'] == severity].iloc[0]
    qpws_row = qpws_severity[qpws_severity['Severity'] == severity].iloc[0]
    
    bais_area = bais_row['Area_km2']
    bais_pct = bais_row['Percentage']
    qpws_area = qpws_row['Area_km2']
    qpws_pct = qpws_row['Percentage']
    
    print(f"| {severity:<13} | {bais_area:>8.2f} | {bais_pct:>8.1f}% | {qpws_area:>8.2f} | {qpws_pct:>8.1f}% |")

total_bais = my_severity_stats['Area_km2'].sum()
print("|---------------|------------|------------|------------|------------|")
print(f"| Total | {total_bais:>8.2f} | 100.0% | {qpws_total_area:>8.2f} | 100.0% |")

# Visualization
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# BAIS2 Severity Distribution
my_severity_stats.plot(kind='bar', x='Class', y='Area_km2', ax=ax1)
ax1.set_title('BAIS2 Severity Distribution')
ax1.set_xlabel('Severity Class')
ax1.set_ylabel('Area (km²)')
ax1.tick_params(axis='x', rotation=45)

# QPWS Severity Distribution
qpws_severity.plot(kind='bar', x='Severity', y='Area_km2', ax=ax2)
ax2.set_title('QPWS Severity Distribution')
ax2.set_xlabel('Severity Class')
ax2.set_ylabel('Area (km²)')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Using the existing severity classes order
severity_order = ['Low', 'Moderate', 'High', 'Extreme']

# Create confusion matrix based on area distributions
def create_area_confusion_matrix(bais2_df, qpws_df):
    # Initialize the confusion matrix
    n_classes = len(severity_order)
    conf_matrix = np.zeros((n_classes, n_classes))
    
    # Calculate the proportional distribution
    total_area = min(bais2_df['Area_km2'].sum(), qpws_df['Area_km2'].sum())
    
    for i, bais2_class in enumerate(severity_order):
        for j, qpws_class in enumerate(severity_order):
            bais2_prop = bais2_df[bais2_df['Class'] == bais2_class]['Area_km2'].iloc[0]
            qpws_prop = qpws_df[qpws_df['Severity'] == qpws_class]['Area_km2'].iloc[0]
            
            # Calculate the overlapping area proportion
            overlap = min(bais2_prop, qpws_prop)
            conf_matrix[i, j] = overlap
    
    return conf_matrix

# Calculate the confusion matrix
conf_matrix = create_area_confusion_matrix(my_severity_stats, qpws_severity)

# Create a DataFrame for better visualization
conf_df = pd.DataFrame(
    conf_matrix,
    index=[f'BAIS2 {cls}' for cls in severity_order],
    columns=[f'QPWS {cls}' for cls in severity_order]
)

# Calculate accuracy metrics
total_area = conf_matrix.sum()
overall_accuracy = np.trace(conf_matrix) / total_area

# Calculate per-class metrics
user_accuracy = np.diag(conf_matrix) / conf_matrix.sum(axis=1)
producer_accuracy = np.diag(conf_matrix) / conf_matrix.sum(axis=0)

# Create accuracy report
accuracy_df = pd.DataFrame({
    'User\'s Accuracy': user_accuracy,
    'Producer\'s Accuracy': producer_accuracy
}, index=severity_order)

# Visualization
plt.figure(figsize=(12, 10))
sns.heatmap(conf_df, annot=True, fmt='.2f', cmap='YlOrRd')
plt.title('Area-based Confusion Matrix: BAIS2 vs QPWS Severity Classifications (km²)')
plt.ylabel('BAIS2 Classification')
plt.xlabel('QPWS Classification')

# Print results
print("\nConfusion Matrix (km²):")
print(conf_df)
print("\nAccuracy Metrics:")
print(f"Overall Accuracy: {overall_accuracy:.2f}")
print("\nPer-class Accuracies:")
print(accuracy_df)

plt.tight_layout()
plt.show()

# Additional visualization: Normalized confusion matrix
plt.figure(figsize=(12, 10))
norm_conf_matrix = conf_matrix / conf_matrix.sum()
norm_conf_df = pd.DataFrame(
    norm_conf_matrix,
    index=[f'BAIS2 {cls}' for cls in severity_order],
    columns=[f'QPWS {cls}' for cls in severity_order]
)

sns.heatmap(norm_conf_df, annot=True, fmt='.2%', cmap='YlOrRd')
plt.title('Normalized Confusion Matrix: BAIS2 vs QPWS Severity Classifications')
plt.ylabel('BAIS2 Classification')
plt.xlabel('QPWS Classification')

plt.tight_layout()
plt.show()

In [ ]:
# Let's examine what we have
print("\nQPWS Data:")
print(type(qpws_projected))
print(qpws_projected.head())
print("\nQPWS CRS:")
print(qpws_projected.crs)

# Try to find BAIS2 spatial data
# List all variables in your workspace that might contain 'bais' or related terms
import gc
variables = [var for var in dir() if not var.startswith('_')]
print("\nAvailable variables:")
for var in variables:
    obj = locals()[var]
    if isinstance(obj, (pd.DataFrame, gpd.GeoDataFrame)):
        print(f"\n{var}:")
        print(type(obj))
        print(obj.head())

In [ ]:
import rasterio
from rasterio import features
import numpy as np
from rasterio.transform import from_bounds
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, cohen_kappa_score
import pandas as pd
from scipy import stats

# First, let's set up our raster parameters
resolution = 20

# Get the bounds of our study area
bounds = qpws_projected.total_bounds
width = int((bounds[2] - bounds[0]) / resolution)
height = int((bounds[3] - bounds[1]) / resolution)
transform = from_bounds(*bounds, width, height)

# Create raster for QPWS data
qpws_raster = np.zeros((height, width))
severity_mapping = {'Low': 1, 'Moderate': 2, 'High': 3, 'Extreme': 4}

# Rasterize QPWS data
shapes = ((geom, severity_mapping[value]) 
          for geom, value in zip(qpws_projected.geometry, qpws_projected.Severity))
qpws_raster = features.rasterize(shapes=shapes, 
                                out_shape=(height, width),
                                transform=transform,
                                fill=0)

# Create raster for BAIS2 data
bais2_raster = features.rasterize(
    [(geom, value) for geom, value in zip(total_burn.geometry, total_burn.attribute)],
    out_shape=(height, width),
    transform=transform,
    fill=np.nan)  # Use NaN for no data

# Improved BAIS2 classification function with proper NaN handling
def classify_bais2(value):
    if np.isnan(value):
        return 0
    elif 0.2 <= value < 0.3:
        return 1  # Low
    elif 0.3 <= value < 0.4:
        return 2  # Moderate
    elif 0.4 <= value < 0.45:
        return 3  # High
    elif value >= 0.45:
        return 4  # Extreme
    else:
        return 0

bais2_classified = np.vectorize(classify_bais2)(bais2_raster)

# Calculate agreement
valid_pixels = (qpws_raster > 0) & (bais2_classified > 0)
agreement = (qpws_raster[valid_pixels] == bais2_classified[valid_pixels]).sum() / valid_pixels.sum()

# Create confusion matrix
class_names = ['Low', 'Moderate', 'High', 'Extreme']
conf_matrix = confusion_matrix(
    qpws_raster[valid_pixels].flatten(),
    bais2_classified[valid_pixels].flatten(),
    labels=[1, 2, 3, 4]
)

# Calculate various accuracy metrics
def calculate_accuracy_metrics(conf_matrix):
    user_acc = np.diag(conf_matrix) / conf_matrix.sum(axis=1)
    prod_acc = np.diag(conf_matrix) / conf_matrix.sum(axis=0)
    overall_acc = np.trace(conf_matrix) / np.sum(conf_matrix)
    
    # Calculate confidence intervals (95%)
    n = np.sum(conf_matrix)
    confidence = 0.95
    interval = stats.norm.ppf(1 - (1 - confidence) / 2) * np.sqrt((overall_acc * (1 - overall_acc)) / n)
    
    # Commission and Omission errors
    commission_errors = 1 - user_acc
    omission_errors = 1 - prod_acc
    
    return {
        'user_accuracy': user_acc,
        'producer_accuracy': prod_acc,
        'overall_accuracy': overall_acc,
        'confidence_interval': interval,
        'commission_errors': commission_errors,
        'omission_errors': omission_errors
    }

metrics = calculate_accuracy_metrics(conf_matrix)

# Create visualizations
plt.figure(figsize=(15, 10))

# 1. Confusion Matrix Heatmap
plt.subplot(1, 2, 1)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='YlOrRd',
            xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix Heatmap')
plt.xlabel('BAIS2 Classification')
plt.ylabel('QPWS Classification')

# 2. Agreement/Disagreement Map
plt.subplot(1, 2, 2)
agreement_map = np.zeros_like(qpws_raster)
agreement_map[valid_pixels] = (qpws_raster[valid_pixels] == bais2_classified[valid_pixels]).astype(int)
plt.imshow(agreement_map, cmap='RdYlGn')
plt.title('Agreement Map (Green: Agreement, Red: Disagreement)')
plt.colorbar(label='Agreement (1) / Disagreement (0)')

plt.tight_layout()
plt.show()

# Print detailed results
print(f"\nOverall Accuracy: {metrics['overall_accuracy']:.2%} (±{metrics['confidence_interval']:.2%})")
print(f"Kappa Coefficient: {cohen_kappa_score(qpws_raster[valid_pixels].flatten(), bais2_classified[valid_pixels].flatten()):.3f}")

print("\nPer-class Metrics:")
for i, name in enumerate(class_names):
    print(f"\n{name} Severity:")
    print(f"User's Accuracy: {metrics['user_accuracy'][i]:.2%}")
    print(f"Producer's Accuracy: {metrics['producer_accuracy'][i]:.2%}")
    print(f"Commission Error: {metrics['commission_errors'][i]:.2%}")
    print(f"Omission Error: {metrics['omission_errors'][i]:.2%}")

# Perform threshold sensitivity analysis
def threshold_sensitivity(bais2_values, qpws_values, threshold_ranges):
    results = []
    for low_thresh in threshold_ranges['low']:
        for mod_thresh in threshold_ranges['moderate']:
            for high_thresh in threshold_ranges['high']:
                if low_thresh < mod_thresh < high_thresh:
                    def temp_classify(value):
                        if np.isnan(value):
                            return 0
                        elif 0.2 <= value < low_thresh:
                            return 1
                        elif low_thresh <= value < mod_thresh:
                            return 2
                        elif mod_thresh <= value < high_thresh:
                            return 3
                        elif value >= high_thresh:
                            return 4
                        else:
                            return 0
                    
                    classified = np.vectorize(temp_classify)(bais2_values)
                    valid = (qpws_values > 0) & (classified > 0)
                    accuracy = (qpws_values[valid] == classified[valid]).mean()
                    results.append({
                        'low_threshold': low_thresh,
                        'moderate_threshold': mod_thresh,
                        'high_threshold': high_thresh,
                        'accuracy': accuracy
                    })
    return pd.DataFrame(results)

# Define threshold ranges to test
threshold_ranges = {
    'low': np.arange(0.25, 0.35, 0.05),
    'moderate': np.arange(0.35, 0.45, 0.05),
    'high': np.arange(0.45, 0.55, 0.05)
}

sensitivity_results = threshold_sensitivity(bais2_raster, qpws_raster, threshold_ranges)
print("\nTop 5 Threshold Combinations:")
print(sensitivity_results.sort_values('accuracy', ascending=False).head())

In [ ]:
import rasterio
from rasterio import features
import numpy as np
from rasterio.transform import from_bounds
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, cohen_kappa_score
import pandas as pd
from scipy import stats
from matplotlib_scalebar.scalebar import ScaleBar

# First, let's set up our raster parameters
resolution = 20

# Get the bounds of our study area
bounds = qpws_projected.total_bounds
width = int((bounds[2] - bounds[0]) / resolution)
height = int((bounds[3] - bounds[1]) / resolution)
transform = from_bounds(*bounds, width, height)

# Create raster for QPWS data
qpws_raster = np.zeros((height, width))
severity_mapping = {'Low': 1, 'Moderate': 2, 'High': 3, 'Extreme': 4}

# Rasterize QPWS data
shapes = ((geom, severity_mapping[value]) 
          for geom, value in zip(qpws_projected.geometry, qpws_projected.Severity))
qpws_raster = features.rasterize(shapes=shapes, 
                                out_shape=(height, width),
                                transform=transform,
                                fill=0)

# Create raster for BAIS2 data
bais2_raster = features.rasterize(
    [(geom, value) for geom, value in zip(total_burn.geometry, total_burn.attribute)],
    out_shape=(height, width),
    transform=transform,
    fill=np.nan)  # Use NaN for no data

# Improved BAIS2 classification function with proper NaN handling
def classify_bais2(value):
    if np.isnan(value):
        return 0
    elif 0.2 <= value < 0.3:
        return 1  # Low
    elif 0.3 <= value < 0.4:
        return 2  # Moderate
    elif 0.4 <= value < 0.45:
        return 3  # High
    elif value >= 0.45:
        return 4  # Extreme
    else:
        return 0

bais2_classified = np.vectorize(classify_bais2)(bais2_raster)

# Calculate agreement
valid_pixels = (qpws_raster > 0) & (bais2_classified > 0)
agreement = (qpws_raster[valid_pixels] == bais2_classified[valid_pixels]).sum() / valid_pixels.sum()

# Create confusion matrix
class_names = ['Low', 'Moderate', 'High', 'Extreme']
conf_matrix = confusion_matrix(
    qpws_raster[valid_pixels].flatten(),
    bais2_classified[valid_pixels].flatten(),
    labels=[1, 2, 3, 4]
)

# Calculate various accuracy metrics
def calculate_accuracy_metrics(conf_matrix):
    user_acc = np.diag(conf_matrix) / conf_matrix.sum(axis=1)
    prod_acc = np.diag(conf_matrix) / conf_matrix.sum(axis=0)
    overall_acc = np.trace(conf_matrix) / np.sum(conf_matrix)
    
    # Calculate confidence intervals (95%)
    n = np.sum(conf_matrix)
    confidence = 0.95
    interval = stats.norm.ppf(1 - (1 - confidence) / 2) * np.sqrt((overall_acc * (1 - overall_acc)) / n)
    
    # Commission and Omission errors
    commission_errors = 1 - user_acc
    omission_errors = 1 - prod_acc
    
    return {
        'user_accuracy': user_acc,
        'producer_accuracy': prod_acc,
        'overall_accuracy': overall_acc,
        'confidence_interval': interval,
        'commission_errors': commission_errors,
        'omission_errors': omission_errors
    }

metrics = calculate_accuracy_metrics(conf_matrix)

# Create improved visualizations
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7))
fig.suptitle('Comparison of QPWS and BAIS2 Burn Severity Classifications', fontsize=14, y=1.02)

# 1. Confusion Matrix Heatmap
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='YlOrRd',
            xticklabels=class_names, yticklabels=class_names, ax=ax1)
ax1.set_title('Confusion Matrix Heatmap')
ax1.set_xlabel('BAIS2 Classification')
ax1.set_ylabel('QPWS Classification')
cbar1 = ax1.collections[0].colorbar
cbar1.set_label('Pixel Count', rotation=270, labelpad=15)

# 2. Improved Agreement Map with proper masking
agreement_map = np.ma.masked_array(np.zeros_like(qpws_raster), mask=True)  # Start with everything masked
valid_area = (qpws_raster > 0) | (bais2_classified > 0)  # Areas where either classification exists
agreement_map.mask[valid_area] = False  # Unmask only valid areas
agreement_map.data[valid_area] = (qpws_raster[valid_area] == bais2_classified[valid_area]).astype(float)

# Create custom colormap for agreement map
colors = ['#d73027', '#1a9850']  # Red for disagreement, Green for agreement
cmap = plt.cm.colors.ListedColormap(colors)

# Plot agreement map
im = ax2.imshow(agreement_map, cmap=cmap, vmin=0, vmax=1)
ax2.set_title('Classification Agreement Map')

# Add custom colorbar
cbar2 = plt.colorbar(im, ax=ax2)
cbar2.set_ticks([0.25, 0.75])
cbar2.set_ticklabels(['Disagreement', 'Agreement'])

# Add axis labels
ax2.set_xlabel('Pixel Column')
ax2.set_ylabel('Pixel Row')

# Add scale bar
scalebar = ScaleBar(resolution, 'm', location='lower left')  # Changed to lower left
ax2.add_artist(scalebar)

# Add North Arrow
ax2.text(0.95, 0.95, '↑N', transform=ax2.transAxes, 
         fontsize=12, fontweight='bold', 
         ha='center', va='center')

plt.tight_layout()
# Save the figure
# Save the figure
plt.savefig('burn_severity_comparison.png', dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# Create figure with subplots
fig = plt.figure(figsize=(15, 12))  # Made figure slightly taller
plt.suptitle('Burn Severity Classification Accuracy Assessment', fontsize=14, y=0.98)  # Moved title up

# Define grid layout with more space between elements
gs = fig.add_gridspec(2, 2, width_ratios=[1, 1.2], height_ratios=[0.3, 1], hspace=0.4)  # Increased height_ratios[0] and hspace

# Overall metrics in top left
ax1 = fig.add_subplot(gs[0, 0])
overall_data = [['Overall Accuracy', f'{41.57}% (±{0.54}%)'],
                ['Kappa Coefficient', f'{0.173:.3f}']]
ax1.axis('tight')
ax1.axis('off')
table1 = ax1.table(cellText=overall_data, 
                  colWidths=[0.5, 0.5],
                  loc='center',
                  cellLoc='left')
table1.auto_set_font_size(False)
table1.set_fontsize(10)
table1.scale(1.2, 1.5)

# Thresholds table in top right
ax2 = fig.add_subplot(gs[0, 1])
threshold_data = pd.DataFrame({
    'Low': [0.25, 0.25, 0.25, 0.25, 0.25],
    'Moderate': [0.35, 0.40, 0.35, 0.40, 0.45],
    'High': [0.55, 0.55, 0.50, 0.50, 0.55],
    'Accuracy': [57.79, 57.30, 56.68, 56.19, 53.58]
})
ax2.axis('tight')
ax2.axis('off')
table2 = ax2.table(cellText=threshold_data.values.round(2),
                  colLabels=['Low', 'Moderate', 'High', 'Accuracy (%)'],
                  loc='center',
                  cellLoc='center')
table2.auto_set_font_size(False)
table2.set_fontsize(10)
table2.scale(1.2, 1.5)
ax2.set_title('Top 5 Threshold Combinations', pad=20)

# Per-class metrics visualization
ax3 = fig.add_subplot(gs[1, :])
class_metrics = pd.DataFrame({
    'Severity Class': ['Low', 'Moderate', 'High', 'Extreme'],
    "User's Accuracy": [76.99, 40.12, 22.27, 88.26],
    "Producer's Accuracy": [27.97, 55.78, 66.16, 21.11],
    'Commission Error': [23.01, 59.88, 77.73, 11.74],
    'Omission Error': [72.03, 44.22, 33.84, 78.89]
})

# Melt the dataframe for easier plotting
melted_metrics = pd.melt(class_metrics, id_vars=['Severity Class'], 
                        var_name='Metric', value_name='Percentage')

# Create grouped bar plot
sns.barplot(x='Severity Class', y='Percentage', hue='Metric', 
            data=melted_metrics, ax=ax3)
ax3.set_ylabel('Percentage (%)')
ax3.set_title('Per-class Accuracy Metrics')
ax3.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.savefig('accuracy_assessment.png', dpi=300, bbox_inches='tight')
plt.show()